In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
import skimage
from scipy.stats import skew, kurtosis
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
from skimage.feature import local_binary_pattern
from dataset.dataset import SkinLesion_Dataset, SegExamples
from pipeline.preprocessing import SkinLesionPreprocessing
from sklearn.model_selection import StratifiedKFold
from statistics import stdev
from sklearn import linear_model
from tqdm import tqdm


preproc = SkinLesionPreprocessing()
examples = SegExamples()

In [ ]:
def lbp_function(paths):
    radius = 1
    n_points = 8 * radius
    lbp_feature_vector=[]
    for i in paths:
        image = cv2.imread(i, cv2.IMREAD_GRAYSCALE) # read image as grayscale
        lbp = local_binary_pattern(image, n_points, radius, method="uniform") # 'uniform' is rotation invariant.
        hist = np.histogram(lbp.ravel())
        feature_array=np.ndarray.tolist(hist[0])
        lbp_feature_vector.append(feature_array)
    return pd.DataFrame(lbp_feature_vector)

In [30]:
# get features example
all_feat = []
labels = []

radius = 1
n_points = 8 * radius
lbp_feature_vector=[]

for i in tqdm(range(examples.seg_examples_df.shape[0])):

    if examples[i]['problem'] != 'binary':
        continue

    labels.append(examples[i]['label'])

    image = examples[i]['img']

    image = preproc.preprocess(image) # the img uploaded as numpy array.

    lbp1 = local_binary_pattern(image[:,:,0], n_points, radius, method="uniform")
    lbp2 = local_binary_pattern(image[:,:,1], n_points, radius, method="uniform")
    lbp3 = local_binary_pattern(image[:,:,2], n_points, radius, method="uniform")

    hist1 = np.histogram(lbp1.ravel())
    hist2 = np.histogram(lbp2.ravel())
    hist3 = np.histogram(lbp3.ravel())

    feature_array1=np.ndarray.tolist(hist1[0])
    feature_array2=np.ndarray.tolist(hist2[0])
    feature_array3=np.ndarray.tolist(hist3[0])

    lbp_all  = np.concatenate((feature_array1, feature_array2, feature_array3),axis=0)
    lbp_feature_vector.append(lbp_all)

d = pd.DataFrame(lbp_feature_vector)
d

 50%|█████     | 1/2 [00:00<00:00,  1.61it/s]

(30,)


100%|██████████| 2/2 [00:01<00:00,  1.11it/s]

(30,)


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17226,22537,15663,24051,25377,27954,24295,25980,32798,48743,...,18363,23626,14430,21422,22713,24918,23420,27381,37376,50975
1,9721,14553,9820,23630,28680,33278,21827,20193,27347,31847,...,12778,17698,8880,18291,17591,25284,21057,23676,35941,39700
